## 深層学習での前処理の順序

分かち書き・正規化
- 単語毎に分割(分かち書き)・正規化(無駄な文字を削除)

ここまでをクレンジング(cleansing)という。

↓

単語埋め込み(Word Embedding)
- 単語をベクトル化

または、

単語をトークン化(Tokenize)
- 単語を一意の ID に変換

↓

パディング(Padding)・端数処理(truncating)

↓

学習モデルに入力

これらは、[言語処理100本ノック 2020 (Rev 2)](https://nlp100.github.io/ja/) にて勉強できる。


カリフォルニア大学アーバイン校がメンテナンスする559個のデータセットからなるリポジトリ

[UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php)

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
!unzip smsspamcollection.zip

In [ ]:
import pandas as pd

df = pd.read_csv('./SMSSpamCollection', sep='\t', header=None)
df.rename({0: 'label', 1: 'text'}, axis=1, inplace=True)
df['category'] = df.apply(lambda r: 1 if r['label'] == 'spam' else 0, axis=1)

df.head()

## 学習データ分割

`sklearn 学習データ 分割`で検索し、`df[['text']], df[['category']]` を `X_train, X_test, Y_train, Y_test` に分割する

分析する**文章**を「**`説明変数`**」、予測する**カテゴリー番号**を「**`目的変数`**」とする。

In [ ]:


# X_train, X_test, Y_train, Y_test の shape(要素数) を見る

## 文章をシーケンス化

keras の `fit_on_texts()` メソッドで「説明変数」のトレインデータとテストデータをベクトル化し、`texts_to_sequences` を使い `x_train`,`x_test`という変数名の行列にする。

## パディング

説明変数のトレインデータとテストデータを`x_train`,`x_test`という変数名でパディングする

## モデル定義

`Sequential`モデルでLSTMを構築する。

語彙数 = `tokenizer.word_index`(インデックス番号)の長さ

`Embedding(vocabulary_size,32)`->`LSTM(16)`->`Dense(1)`

`compile`で

- 損失関数：バイナリクロスエントロピー
- 最適化アルゴリズム：Adam(学習率=0.001)
- 評価関数のリスト：`['accuracy']`

`summary()`でモデルを可視化

## 学習

目的変数の値をトレイン・テストとも`.values`でデータフレームから配列に変換する。

`fit()`で学習を行う。
説明変数と目的変数のトレインデータ、バッチサイズ(32)、エポック数(5)、検証データ(損失とモデル評価関数の性能を評価する)として説明変数と目的変数をいれる

## モデル評価

### サンプルを入力してみる
サンプルテキストの配列を作り、ベクトル化せずシーケンス化、パディングする。
`predict_classes()`に入力すると評価できる。

In [ ]:
texts = [
    "Are you single? Use this site to find women in their 20s who want to have sex right away! Pay $ 100 Now and Get FREE! 08717890890",
    "Are you single? Use this site to find women in their 20s who want to have sex right away! Pay $ 100 Now and Get FREE! 08717890890 http://porn.sex/xxx.asp?o=13543",
    "Hello. It is nice weather. It's time to see the cherry blossoms, so how about cherry blossom viewing together"
]

### 精度、再現率、F値を計算

`predict()`で入力サンプルに対する予測値の出力を生成する。説明変数のテストデータを入力。

`confusion_matrix()`で

混合行列 = 目的変数のテストデータと予測値

tn, fp, fn, tp = 目的変数のテストデータと予測値 + ravel()メソッド

In [ ]:


print("精度(Precision): {:.2f}%".format(100 * precision_score(y_test, y_predict.round())))
print("再現率(Recall): {:.2f}%".format(100 * recall_score(y_test, y_predict.round())))
print("F値(F1 Score): {:.2f}%".format(100 * f1_score(y_test,y_predict.round())))

### ヒートマップで混合行列を図示する

`sns.heatmap(混合行列、annot=True(値を表示), cmap='Blue'(配色を青色に), fmt='d'(値を整数))`

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
ax= plt.subplot()

#annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels');
ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');
ax.xaxis.set_ticklabels(['Not Spam', 'Spam']); ax.yaxis.set_ticklabels(['Not Spam', 'Spam']);